In [1]:
import graphlab

In [2]:
sales = graphlab.SFrame('course-2/kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

[INFO] This non-commercial license of GraphLab Create is assigned to srb1706@gmail.com and will expire on March 07, 2017. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-1893 - Server binary: /usr/local/lib/python2.7/dist-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1459035637.log
[INFO] GraphLab Server Version: 1.8


In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # add a constant column to an SFrame
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’
    features_sframe = data_sframe[features]
    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’
    output_sarray = data_sframe[output]
    # this will convert the SArray into a numpy array:
    output_array = output_sarray.to_numpy() # GraphLab Create>= 1.7!!
    return(features_matrix, output_array)

In [5]:
def predict_outcome(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

In [11]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    return(feature_matrix/norms,norms)

In [12]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


In [13]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [14]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [15]:
weights = np.array([1., 4., 1.])

In [16]:
prediction = predict_outcome(simple_feature_matrix,weights)

In [23]:
ro = []
for i in range(len(weights)):
    ro.append(sum(simple_feature_matrix[:,i]*(output - prediction + weights[i]*simple_feature_matrix[:,i])))

In [24]:
ro

[79400300.034929529, 87939470.772990793, 80966698.67596525]

In [38]:

def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    norm_feature_matrix, norms = normalize_features(feature_matrix)
    prediction = predict_outcome(norm_feature_matrix, weights)
#    print("feature_matrix: %s" % feature_matrix)
#    print("norm_feature_matrix: %s" % norm_feature_matrix)
    
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    feature_i = norm_feature_matrix[:,i]
    tmp = feature_i * (output - prediction + weights[i]*feature_i)
    ro_i = tmp.sum()
#    print "ro_i: %f" % ro_i
#    print "l1_penalty: %f" % l1_penalty
    
    #        ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
    # w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
    #        └ (ro[i] - lambda/2)     if ro[i] > lambda/2
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [39]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


In [40]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance, verbose=False):
    if verbose is True:
        print("tolerance: %f" % tolerance)
    weights = initial_weights
    loop_max = 1000
    for n in range(loop_max):
        need_continue = False
        for i in range(len(weights)):
            old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
            # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
            #     and old values for weight[i], ..., weight[d-1]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)

            # use old_weights_i to compute change in coordinate
            change = abs(weights[i] - old_weights_i)
            if verbose is True:
                print("change: %f" % change)
            if change > tolerance:
                need_continue = True
        if verbose is True:
            print("[%d] need continue? %s" % (n, need_continue))
        if need_continue is False:
            break
    return weights

In [41]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [42]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

In [43]:

weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [45]:
print normalized_simple_feature_matrix
print weights
predict = predict_outcome(normalized_simple_feature_matrix, weights)
print predict
print output
tmp = (predict - output) ** 2
print tmp
print "rss: %f" % tmp.sum()
print "weights: %s" % weights

[[ 0.00680209  0.00353021  0.00583571]
 [ 0.00680209  0.00768869  0.00583571]
 [ 0.00680209  0.00230361  0.00389048]
 ..., 
 [ 0.00680209  0.00305154  0.00389048]
 [ 0.00680209  0.00478673  0.00583571]
 [ 0.00680209  0.00305154  0.00389048]]
[ 21624998.36638625  63157246.7854307          0.        ]
[ 370053.87731146  632691.61911814  292585.19087927 ...,  339822.19480133
  449412.04390053  339822.19480133]
[ 221900.  538000.  180000. ...,  402101.  400000.  325000.]
[  2.19495714e+10   8.96650273e+09   1.26754252e+10 ...,   3.87864958e+09
   2.44155008e+09   2.19697459e+08]
rss: 1630492481484728.000000
weights: [ 21624998.36638625  63157246.7854307          0.        ]


In [46]:
train_data,test_data = sales.random_split(.8,seed=0)

In [47]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [48]:
my_output = 'price'
(all_feature_matrix, output) = get_numpy_data(sales, all_features, my_output)
(normalized_all_feature_matrix, all_norms) = normalize_features(all_feature_matrix) # normalize features

In [54]:
l1_penalty = 1e7
initial_weights = np.zeros(len(all_features)+1)
tolerance = 1.0
print initial_weights
weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print weights1e7

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 24964803.20413724         0.                 0.          56397533.12094329
         0.                 0.           3689656.60016693
   8630251.00034065         0.                 0.                 0.
         0.                 0.                 0.        ]


In [50]:
l1_penalty = 1e8
weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print "weights1e8: %s" % weights1e8

weights1e8: [ 79400304.6580555         0.                0.                0.
         0.                0.                0.                0.
         0.                0.                0.                0.
         0.                0.       ]


In [51]:

l1_penalty = 1e4
weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print "weights1e4: %s" % weights1e4

weights1e4: [  1.60222157e+08  -1.82014152e+07   2.20561544e+06   2.04839630e+08
  -2.07064219e+06  -3.33639433e+06   7.21432356e+06   6.45538318e+06
   2.26664724e+07   1.25347193e+08  -1.24113892e+08  -3.25961158e+07
  -2.67812200e+08   3.31613270e+06]


In [52]:

normalized_weights1e4 = weights1e4 / all_norms
normalized_weights1e7 = weights1e7 / all_norms
normalized_weights1e8 = weights1e8 / all_norms
print normalized_weights1e7[3]

612.820280434
